In [47]:
import pandas as pd
from datetime import datetime,timedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

product = pd.read_csv('data/dim_product.csv',index_col=None)
store_stock =  pd.read_csv('data/store_stock.csv',index_col=None)
warehouse_stock =  pd.read_csv('data/warehouse_stock.csv',index_col=None)
predictions = pd.read_csv('data/daily_breakdown_store.csv',index_col=None)
sales =  pd.read_csv('data/history_sales.csv',index_col=None)
orders = pd.read_csv('data/open_order.csv',index_col=None)

C:\Users\FromHell\AppData\Local\Temp\ipykernel_10972\844458542.py:7: DtypeWarning: Columns (1,22) have mixed types. Specify dtype option on import or set low_memory=False.
  product = pd.read_csv('data/dim_product.csv',index_col=None)
C:\Users\FromHell\AppData\Local\Temp\ipykernel_10972\844458542.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  store_stock =  pd.read_csv('data/store_stock.csv',index_col=None)
C:\Users\FromHell\AppData\Local\Temp\ipykernel_10972\844458542.py:11: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  sales =  pd.read_csv('data/history_sales.csv',index_col=None)


In [48]:
orders['acik_siparis'] = orders['toplam_yoldaki_miktar'] + orders['toplanmayan_urun']
orders.rename(columns={'depo_kodu' : 'store_code'},inplace=True)
orders.head()

,product_id,store_code,toplam_yoldaki_miktar,toplanmayan_urun,acik_siparis
0,000LP-0002Dark Indigo - Worn In34-30,9001,1,0,1
1,000RY-0000WhitesM,2801,1,0,1
2,000RY-0002BlacksM,2701,1,0,1
3,001T2-0010Light Indigo - Worn In32-30,9001,1,0,1
4,001T2-0010Light Indigo - Worn In34-32,9001,1,0,1


In [49]:
product_segments_cols = [
    'product_id',
    'product_att_01', 'product_att_02', 'product_att_03'
]
sales_with_segment = sales.merge(product[product_segments_cols], on='product_id', how='left')

sales_with_segment['product_att_02'] = sales_with_segment['product_att_02'].apply(
    lambda x: int(str(x).replace('D', '9').replace('N', '9')) if isinstance(x, str) and ('D' in x or 'N' in x) else x
)

sales_with_segment['product_att_02'] = pd.to_numeric(sales_with_segment['product_att_02'], errors='coerce')


# Filtrele: 9.0'lar hariç
filtered_sales = sales_with_segment[
    (~(sales_with_segment['product_att_01'] == 9.0)) &
    (~(sales_with_segment['product_att_02'] == 9.0))
]
filtered_sales = sales_with_segment[
    (~(sales_with_segment['product_att_01'] == 999)) &
    (~(sales_with_segment['product_att_02'] == 999))
]

filtered_sales = filtered_sales[filtered_sales["net_quantity"] >= 0]
filtered_sales['date'] = pd.to_datetime(filtered_sales['date'])
sales = filtered_sales



In [50]:
store_stock['product_id'] = store_stock['stok_kodu'] + store_stock['color_code'] + store_stock['size'] 
store_stock_agg = store_stock.groupby(
    ['stok_tarihi','magaza','stok_kodu','product_id']
).agg({'toplam_miktar':'sum'}).reset_index()
store_stock_agg.rename(columns={'toplam_miktar': 'stok_miktar'}, inplace=True)


store_stock_agg['stok_miktar'] = store_stock_agg['stok_miktar'].astype(int)
store_stock_agg.head()

,stok_tarihi,magaza,stok_kodu,product_id,stok_miktar
0,2025-08-16,201,04146501,04146501Puma Black/Gray VioletL,1
1,2025-08-16,201,04146603,04146603Puma BlackL,1
2,2025-08-16,201,04164101,04164101Puma Black/Gold/Puma White10,2
3,2025-08-16,201,04164101,04164101Puma Black/Gold/Puma White11,2
4,2025-08-16,201,04164101,04164101Puma Black/Gold/Puma White4,2


# Agırlıklı Satıslar

In [51]:
max_date = pd.to_datetime(sales['date'].max())
start_7 = max_date - pd.Timedelta(days=6)  

mask = (sales['date'] >= start_7) & (sales['date'] <= max_date)
sales_agg_7 = sales.loc[mask]

sales_7 = (sales_agg_7
           .groupby(['product_id','store_code'], as_index=False)['net_quantity']
           .sum()
           .rename(columns={'net_quantity':'net_quantity_7'}))

start_14 = max_date - pd.Timedelta(days=13)  

mask = (sales['date'] >= start_14) & (sales['date'] <= max_date)
sales_agg_14 = sales.loc[mask]

sales_14 = (sales_agg_14
           .groupby(['product_id','store_code'], as_index=False)['net_quantity']
           .sum()
           .rename(columns={'net_quantity':'net_quantity_14'}))

start_28 = max_date - pd.Timedelta(days=27)  

mask = (sales['date'] >= start_28) & (sales['date'] <= max_date)
sales_agg_28 = sales.loc[mask]

sales_28 = (sales_agg_28
           .groupby(['product_id','store_code'], as_index=False)['net_quantity']
           .sum()
           .rename(columns={'net_quantity':'net_quantity_28'}))



# ürün-mağaza ikilisi belirleme

In [52]:

def get_store_sku_pairs_union(
    store_stock: pd.DataFrame,
    prediction: pd.DataFrame | None = None,
    # stok tarafı
    stock_date_col="stok_tarihi",
    store_col="magaza",
    sku_col="stok_kodu",
    stock_qty_col="stok_miktar",
    lookback_stock_days=30,
    # prediction tarafı 
    pred_date_col="date",
    pred_store_col="store_code",
    pred_sku_col="stok_kodu",
    pred_qty_col="store_predicted_quantity",
    lookahead_pred_days=30,
    min_pred=0.0
):

    ss = store_stock_agg.copy()
    ss[stock_date_col] = pd.to_datetime(ss[stock_date_col])
    ss[store_col] = ss[store_col].astype(str)
    ss[sku_col] = ss[sku_col].astype(str)
    ss[stock_qty_col] = pd.to_numeric(ss[stock_qty_col], errors="coerce").fillna(0)


    max_dates = [ss[stock_date_col].max()]
    if prediction is not None:
        max_dates.append(pd.to_datetime(prediction[pred_date_col]).max())
    today = max([d for d in max_dates if pd.notna(d)])

    # son 30 günde pozitif stok
    start_stock = today - pd.Timedelta(days=lookback_stock_days-1)
    stock_pairs = (
        ss[(ss[stock_date_col] >= start_stock) & (ss[stock_date_col] <= today) & (ss[stock_qty_col] > 0)]
        [[store_col, sku_col]]
        .drop_duplicates()
    )

    # prediction’dan gelen ikililer 
    if prediction is not None:
        pr = prediction.copy()
        pr[pred_date_col] = pd.to_datetime(pr[pred_date_col])
        pr[pred_store_col] = pr[pred_store_col].astype(str)
        pr[pred_sku_col] = pr[pred_sku_col].astype(str)
        pr[pred_qty_col] = pd.to_numeric(pr[pred_qty_col], errors="coerce").fillna(0)


        end_pred   = today + pd.Timedelta(days=lookahead_pred_days-1)

        pred_pairs = (
            pr[(pr[pred_date_col] <= end_pred) & (pr[pred_qty_col] > min_pred)]
            [[pred_store_col, pred_sku_col]]
            .drop_duplicates()
            .rename(columns={pred_store_col: store_col, pred_sku_col: sku_col})
        )
        pairs = pd.concat([stock_pairs, pred_pairs], ignore_index=True).drop_duplicates()
    else:
        pairs = stock_pairs

    return pairs.sort_values([store_col, sku_col]).reset_index(drop=True)
store_sku_pairs = get_store_sku_pairs_union(store_stock)
store_sku_pairs


,magaza,stok_kodu
0,1101,04146501
1,1101,04146603
2,1101,04164101
3,1101,04177301
4,1101,04177308
...,...,...
41202,701,980370-3325
41203,701,980377-2001
41204,701,FC-3007
41205,701,FC-3008


In [53]:
store_sku_pairs["magaza"]  = store_sku_pairs["magaza"].astype(str)
store_sku_pairs["stok_kodu"] = store_sku_pairs["stok_kodu"].astype(str)

product["product_code"] = product["product_code"].astype(str)
product["product_id"] = product["product_id"].astype(str)

pm_active = product.copy()
if "is_blocked" in pm_active.columns:
    pm_active = pm_active.loc[pm_active["is_blocked"].fillna(0).astype(int) == 0]

store_product_pairs = (
    store_sku_pairs
    .merge(pm_active[["product_code", "product_id"]], left_on="stok_kodu", right_on="product_code", how="left")
    .drop(columns=["product_code"])
    .drop_duplicates()
    .reset_index(drop=True)
)
store_product_pairs.rename({'magaza':'store_code'}, axis=1, inplace=True)

store_product_pairs.count()

store_code    241006
stok_kodu     241006
product_id    241006
dtype: int64

# safety stock

In [54]:
import pandas as pd
import numpy as np
from scipy.stats import norm

def seasonal_safety_stock(
    sales: pd.DataFrame,
    date_col="date",
    store_col="store_code",          
    product_col="product_id",
    qty_col="net_quantity",
    lead_time_days=2,
    service_level=0.9,
    month_window=1                   
):
    df = sales.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df[product_col] = df[product_col].astype(str)
    df[qty_col] = pd.to_numeric(df[qty_col], errors="coerce").fillna(0)
    df[store_col] = df[store_col].astype(str)   # <-- mağaza tipi

    today = df[date_col].max()
    current_month = today.month

    # Mevsimsel filtre
    months_to_keep = [(current_month + i - 1) % 12 + 1 for i in range(-month_window, month_window + 1)]
    df_seasonal = df[df[date_col].dt.month.isin(months_to_keep)]

    # Günlük talep (STORE + PRODUCT + DATE)
    daily_demand = (
        df_seasonal.groupby([store_col, product_col, date_col], as_index=False)[qty_col]
                   .sum()
    )

    # Standart sapma ve ortalama (STORE + PRODUCT)
    demand_stats = (
        daily_demand.groupby([store_col, product_col])
        .agg(
            daily_avg=(qty_col, "mean"),
            daily_std=(qty_col, "std")
        )
        .fillna(0)
        .reset_index()
    )

    # Z değeri
    z_value = norm.ppf(service_level)

    # Safety stock
    demand_stats["safety_stock"] = (
        z_value * demand_stats["daily_std"] * np.sqrt(lead_time_days)
    ).round(2)

    return demand_stats

# Örnek kullanım (kolon adın "store_Code" ise store_col="store_Code" yaz):
ss_df = seasonal_safety_stock(sales, store_col="store_code", month_window=1)
ss_df


,store_code,product_id,daily_avg,daily_std,safety_stock
0,101,00501-3184THUNDER MOON ROCKS32-30,2.000000,0.000000,0.00
1,101,00501-3285CANYON MILD34-32,2.000000,0.000000,0.00
2,101,00514-1422DARK HOLLOW34-32,2.000000,0.000000,0.00
3,101,015M100031-NBK-CHOCHOCOLATE // NUBUK43,4.000000,0.000000,0.00
4,101,015M101226-NBK-BLKBLACK // NUBUK40,2.000000,0.000000,0.00
...,...,...,...,...,...
231890,701,FC-3007STDSTD,1.625000,1.767767,3.20
231891,701,FC-3008STDSTD,1.333333,0.577350,1.05
231892,701,FC-4022STDSTD,1.600000,1.352247,2.45
231893,701,T21216-2064GREY8,3.000000,1.414214,2.56


# option agırlıklandırma

In [55]:
LOOKBACK_SALES = 90
LOOKBACK_STOCK = 60

# prediction varsa ondan, yoksa sales'ten "bugün"
end_date = pd.to_datetime(sales["date"]).max()

# Tipler
sales = sales.copy()
sales["date"] = pd.to_datetime(sales["date"])
for c in ["store_code", "stok_kodu", "product_id"]:
    sales[c] = sales[c].astype(str)

pairs = store_product_pairs.rename(columns={"magaza": "store_code"}).copy()
for c in ["store_code", "stok_kodu", "product_id"]:
    pairs[c] = pairs[c].astype(str)

stock = store_stock.copy()
stock["stok_tarihi"] = pd.to_datetime(stock["stok_tarihi"])
for c in ["magaza", "stok_kodu", "product_id"]:
    stock[c] = stock[c].astype(str)

start_sales = end_date - pd.Timedelta(days=LOOKBACK_SALES - 1)
s_win = sales[(sales["date"] >= start_sales) & (sales["date"] <= end_date)]
s_win = s_win[s_win["net_quantity"] > 0]

# Sadece mevcut pair + product_id 
s_win = s_win.merge(pairs[["store_code", "stok_kodu", "product_id"]],
                    on=["store_code", "stok_kodu", "product_id"], how="inner")

sales_agg = (
    s_win.groupby(["store_code", "stok_kodu", "product_id"], as_index=False)["net_quantity"]
         .sum()
         .rename(columns={"net_quantity": "sales_90d"})
)

sales_agg["weight_sales"] = (
    sales_agg["sales_90d"] /
    sales_agg.groupby(["store_code", "stok_kodu"])["sales_90d"].transform("sum")
)

start_stock = end_date - pd.Timedelta(days=LOOKBACK_STOCK - 1)
st_win = stock[(stock["stok_tarihi"] >= start_stock) & (stock["stok_tarihi"] <= end_date)]
st_win = st_win[st_win["toplam_miktar"] > 0]

# store_code isim eşitlemesi
st_win = st_win.rename(columns={"magaza": "store_code"})

st_win = st_win.merge(pairs[["store_code", "stok_kodu", "product_id"]],
                      on=["store_code", "stok_kodu", "product_id"], how="inner")

stock_agg = (
    st_win.groupby(["store_code", "stok_kodu", "product_id"], as_index=False)["toplam_miktar"]
         .sum()
         .rename(columns={"toplam_miktar": "onhand_30d"})
)

stock_agg["weight_onhand"] = (
    stock_agg["onhand_30d"] /
    stock_agg.groupby(["store_code", "stok_kodu"])["onhand_30d"].transform("sum")
)
# (pair + product_id)
children = pairs.copy()  # store_code, stok_kodu, product_id

weights = (children
    .merge(sales_agg[["store_code","stok_kodu","product_id","weight_sales"]], 
           on=["store_code","stok_kodu","product_id"], how="left")
    .merge(stock_agg[["store_code","stok_kodu","product_id","weight_onhand"]],
           on=["store_code","stok_kodu","product_id"], how="left")
)

# Öncelik: sales -> onhand
weights["weight"] = weights["weight_sales"].where(
    weights["weight_sales"].notna(), weights["weight_onhand"]
)

# Tamamı NaN olan grupları eşit böl
weights["n_option"] = weights.groupby(["store_code","stok_kodu"])["product_id"].transform("count")
grp_has_any = weights.groupby(["store_code","stok_kodu"])["weight"].transform(lambda s: s.notna().any())
weights.loc[~grp_has_any, "weight"] = 1.0 / weights["n_option"]

# Normalizasyon (toplam = 1)
sum_w = weights.groupby(["store_code","stok_kodu"])["weight"].transform("sum")
# Eğer nadiren sum_w=0 kalırsa, eşit böl:
zero_sum = (sum_w == 0)
weights.loc[zero_sum, "weight"] = 1.0 / weights.loc[zero_sum, "n_option"]
sum_w = weights.groupby(["store_code","stok_kodu"])["weight"].transform("sum")
weights["weight"] = weights["weight"] / sum_w

weight_map = weights[["store_code","stok_kodu","product_id","weight"]].copy()
weight_map


,store_code,stok_kodu,product_id,weight
0,1101,04146501,04146501Puma Black/Gray VioletL,0.500000
1,1101,04146501,04146501Puma Black/Gray VioletM,0.500000
2,1101,04146603,04146603Puma BlackL,0.500000
3,1101,04146603,04146603Puma BlackM,0.500000
4,1101,04164101,04164101Puma Black/Gold/Puma White10,0.285714
...,...,...,...,...
241001,701,980370-3325,980370-3325LILAS111,1.000000
241002,701,980377-2001,980377-2001BLACK111,1.000000
241003,701,FC-3007,FC-3007STDSTD,1.000000
241004,701,FC-3008,FC-3008STDSTD,1.000000


# prediction agırlıklandırma

In [56]:
predictions = predictions[predictions["store_predicted_quantity"] >= 0]
pred = predictions.copy()  
pred["date"] = pd.to_datetime(pred["date"])
pred.rename({'date':'sales_date'},axis=1, inplace=True)
pred["store_code"] = pred["store_code"].astype(str)
pred["stok_kodu"] = pred["stok_kodu"].astype(str)
pred["store_predicted_quantity"] = pd.to_numeric(pred["store_predicted_quantity"], errors="coerce").fillna(0)
pred_pid = (
    pred.merge(weight_map, on=["store_code", "stok_kodu"], how="inner")
)
pred_pid["prediction_df"] = pred_pid["store_predicted_quantity"] * pred_pid["weight"]

pred_pid = pred_pid[["store_code", "stok_kodu", "product_id", "sales_date", "prediction_df"]]
pred_pid['prediction_df'] = pred_pid['prediction_df'].fillna(0).astype(float)
pred_pid

,store_code,stok_kodu,product_id,sales_date,prediction_df
0,301,38073866,38073866Puma Black/Puma White/Modern Mint40.5,2025-07-07,0.000000
1,301,38073866,38073866Puma Black/Puma White/Modern Mint41,2025-07-07,0.015873
2,301,38073866,38073866Puma Black/Puma White/Modern Mint42,2025-07-07,0.015873
3,301,38073866,38073866Puma Black/Puma White/Modern Mint42.5,2025-07-07,0.095238
4,301,38073866,38073866Puma Black/Puma White/Modern Mint43,2025-07-07,0.015873
...,...,...,...,...,...
5117443,2801,SP2K-0029,SP2K-0029Multi-ColorD,2025-09-24,0.116093
5117444,2801,SP2K-0029,SP2K-0029Multi-ColorD,2025-09-25,0.116093
5117445,2801,SP2K-0029,SP2K-0029Multi-ColorD,2025-09-26,0.116093
5117446,2801,SP2K-0029,SP2K-0029Multi-ColorD,2025-09-27,0.116093


In [57]:
base_calc_table_0 = store_product_pairs.merge(
    ss_df[["product_id", "store_code", "safety_stock"]],
    on=["product_id", "store_code"],
    how="left"

)
base_calc_table_0["safety_stock"] = pd.to_numeric(base_calc_table_0["safety_stock"], errors="coerce").fillna(0)
base_calc_table_0['store_code']=base_calc_table_0['store_code'].astype(int)

In [58]:
max_date = pd.to_datetime(filtered_sales['date'].max())
end_date = max_date + timedelta(14)
prediction_daily = pred_pid[(pred_pid['sales_date']<= end_date)&(pred_pid['sales_date'] > max_date)]
prediction_daily = prediction_daily.groupby(
    ['store_code','stok_kodu','product_id']
)['prediction_df'].sum().reset_index()
prediction_daily['store_code'] = prediction_daily['store_code'].astype(int)

base_calc_table_1 = base_calc_table_0.merge(
    prediction_daily[["product_id", "store_code", "prediction_df"]],
    on=["product_id", "store_code"],
    how="left"

)
base_calc_table_1['prediction_df'] = base_calc_table_1['prediction_df'].fillna(0).astype(float)
base_calc_table_1.head()

,store_code,stok_kodu,product_id,safety_stock,prediction_df
0,1101,04146501,04146501Puma Black/Gray VioletL,0.0,0.000000
1,1101,04146501,04146501Puma Black/Gray VioletM,0.0,0.000000
2,1101,04146603,04146603Puma BlackL,0.0,0.000000
3,1101,04146603,04146603Puma BlackM,0.0,0.000000
4,1101,04164101,04164101Puma Black/Gold/Puma White10,0.0,0.804232


In [59]:
store_stock_agg.rename(columns={'magaza':'store_code'}, inplace=True)

base_calc_table_2 = base_calc_table_1.merge(
    store_stock_agg[["product_id", "store_code", "stok_miktar"]],
    on=["product_id", "store_code"],
    how="left"

)
base_calc_table_2['stok_miktar'] = base_calc_table_2['stok_miktar'].fillna(0).astype(int)



In [60]:

base_calc_table_3 = (base_calc_table_2
    .merge(sales_7,  on=['store_code','product_id'], how='left')
    .merge(sales_14, on=['store_code','product_id'], how='left')
    .merge(sales_28, on=['store_code','product_id'], how='left')
)

for c in ['net_quantity_7','net_quantity_14','net_quantity_28']:
    base_calc_table_3[c] = base_calc_table_3[c].fillna(0).astype(int)
base_calc_table_3.head()


,store_code,stok_kodu,product_id,safety_stock,prediction_df,stok_miktar,net_quantity_7,net_quantity_14,net_quantity_28
0,1101,04146501,04146501Puma Black/Gray VioletL,0.0,0.000000,1,0,0,0
1,1101,04146501,04146501Puma Black/Gray VioletM,0.0,0.000000,0,0,0,0
2,1101,04146603,04146603Puma BlackL,0.0,0.000000,1,0,0,0
3,1101,04146603,04146603Puma BlackM,0.0,0.000000,0,0,0,0
4,1101,04164101,04164101Puma Black/Gold/Puma White10,0.0,0.804232,2,0,0,0


In [61]:
base_calc_table_3['prediction_ma'] = (
    (base_calc_table_3['net_quantity_7']/7)*0.5 + 
    (base_calc_table_3['net_quantity_14']/14)*0.3 + 
    (base_calc_table_3['net_quantity_28']/28)*0.2
)*14
base_calc_table_3.head(10)

,store_code,stok_kodu,product_id,safety_stock,prediction_df,stok_miktar,net_quantity_7,net_quantity_14,net_quantity_28,prediction_ma
0,1101,04146501,04146501Puma Black/Gray VioletL,0.0,0.000000,1,0,0,0,0.0
1,1101,04146501,04146501Puma Black/Gray VioletM,0.0,0.000000,0,0,0,0,0.0
2,1101,04146603,04146603Puma BlackL,0.0,0.000000,1,0,0,0,0.0
3,1101,04146603,04146603Puma BlackM,0.0,0.000000,0,0,0,0,0.0
4,1101,04164101,04164101Puma Black/Gold/Puma White10,0.0,0.804232,2,0,0,0,0.0
5,1101,04164101,04164101Puma Black/Gold/Puma White11,0.0,0.402116,2,0,1,1,0.4
6,1101,04164101,04164101Puma Black/Gold/Puma White4,0.0,0.000000,1,0,0,0,0.0
7,1101,04164101,04164101Puma Black/Gold/Puma White5,0.0,0.000000,1,0,0,0,0.0
8,1101,04164101,04164101Puma Black/Gold/Puma White6,0.0,0.804232,1,0,0,0,0.0
9,1101,04164101,04164101Puma Black/Gold/Puma White7,0.0,0.000000,3,0,0,0,0.0


In [ ]:
base_calc_table_4 = (
    base_calc_table_3
    .merge(
        orders[['product_id','store_code', 'acik_siparis']], 
        on=['store_code','product_id'], 
        how='left'
    )
)
base_calc_table_4['acik_siparis'] = base_calc_table_4['acik_siparis'].fillna(0).astype(int)


In [75]:
def need_with_14day_coverage(
    df: pd.DataFrame,
    pred_df_col="prediction_df",     # 14g model tahmini
    pred_ma_col="prediction_ma",     # 14g MA tahmini
    ss_col="safety_stock",
    onhand_col="stok_miktar",
    last14_col="net_quantity_14",    # son 14 gün satış
    last28_col="net_quantity_28",    # son 28 gün satış
    lead_time_days=2,
    in_transit_col='acik_siparis',             # varsa "in_transit" gibi bir kolon adı ver
    w_min=0.25, w_max=0.75           # model ağırlığı sınırı
) -> pd.DataFrame:
    out = df.copy()

    # sayısal kolonları temizle
    for c in [pred_df_col, pred_ma_col, ss_col, onhand_col, last14_col, last28_col]:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0.0)
        else:
            out[c] = 0.0
    if in_transit_col and in_transit_col in out.columns:
        out[in_transit_col] = pd.to_numeric(out[in_transit_col], errors="coerce").fillna(0.0)
    else:
        in_transit_col = None


    avg14_from_28 = out[last28_col] / 2.0
    with np.errstate(divide="ignore", invalid="ignore"):
        momentum = out[last14_col] / avg14_from_28.replace(0, np.nan)
    momentum = momentum.fillna(1.0).clip(0.0, 2.0)

    w_model = (0.5 + 0.25 * (momentum - 1.0)).clip(w_min, w_max)  # 0.25..0.75
    # koruyucu kurallar
    w_model = np.where((out[pred_ma_col] <= 0) & (out[pred_df_col] > 0), 1.0, w_model)
    w_model = np.where((out[pred_df_col] <= 0) & (out[pred_ma_col] > 0) & (out[last28_col] > 0), 0.0, w_model)
    out["w_model"] = w_model.astype(float)

    out["pred_14_blended"] = out["w_model"] * out[pred_df_col] + (1.0 - out["w_model"]) * out[pred_ma_col]

    out["pred_LT"] = out["pred_14_blended"] * (lead_time_days / 14.0)

    out["OUL_14"] = out[ss_col] + out["pred_14_blended"]
    expected_onhand_at_arrival = out[onhand_col] - out["pred_LT"]
    if in_transit_col:
        expected_onhand_at_arrival = expected_onhand_at_arrival + out[in_transit_col]
    out["ihtiyac_adet"] = np.ceil(np.maximum(0.0, out["OUL_14"] - expected_onhand_at_arrival)).astype(int)

    return out

In [86]:
result = need_with_14day_coverage(base_calc_table_4, lead_time_days=2)
result.to_csv(
    "output/replenishment_results.csv",
    index=False,
    sep=";",             
    encoding="utf-8-sig",
    float_format="%.6f"  
)

In [104]:
x.groupby(['store_code','date'], as_index=False)['net_quantity'] \
 .sum() \
 .sort_values('date', ascending=False)


,store_code,date,net_quantity
472,1601,2025-02-07,14.0
471,1601,2025-02-06,2.0
470,1601,2025-02-04,4.0
469,1601,2025-02-03,2.0
468,1601,2025-02-02,16.0
467,1601,2025-02-01,20.0
466,1601,2025-01-31,10.0
465,1601,2025-01-30,8.0
464,1601,2025-01-29,8.0
463,1601,2025-01-28,14.0


In [ ]:
import pandas as pd
from collections import defaultdict

def allocate_round_robin(
    demand_df: pd.DataFrame,   
    wh_df: pd.DataFrame,       
    keys=("stok_kodu","product_id"),
    order_by=("ihtiyac_adet",) 
):
    dem = demand_df.copy()
    wh  = wh_df.copy()

    # ---- tip & filtre ----
    for c in ["store_code", keys[0], keys[1]]:
        dem[c] = dem[c].astype(str).str.strip()
    dem["ihtiyac_adet"] = pd.to_numeric(dem["ihtiyac_adet"], errors="coerce").fillna(0).astype(int)
    dem = dem[dem["ihtiyac_adet"] > 0].copy()

    for c in ["alt_yer", keys[0], keys[1]]:
        wh[c] = wh[c].astype(str).str.strip()
    wh["miktar"] = pd.to_numeric(wh["miktar"], errors="coerce").fillna(0).astype(int)
    wh = wh[wh["miktar"] > 0].copy()

    # ---- depo stoklarını SKU bazında topla ----
    wh_agg = (wh.groupby(["alt_yer", *keys], as_index=False)["miktar"]
                .sum().rename(columns={"miktar":"available"}))

    allocations = []  # alt_yer, store_code, keys..., alloc_qty
    used_totals = defaultdict(int)

    # ---- her depo × SKU için 1'er 1'er dağıt ----
    for (alt_yer, sku, pid), wrow in wh_agg.set_index(["alt_yer", *keys]).iterrows():
        avail = int(wrow["available"])
        if avail <= 0:
            continue

        needers = dem[(dem[keys[0]] == sku) & (dem[keys[1]] == pid) & (dem["ihtiyac_adet"] > 0)].copy()
        if needers.empty:
            continue

        # sıralama: ihtiyac_adet DESC, (eşitlikte) store_code ASC
        if order_by:
            by = list(order_by) + ["store_code"]
            asc = [False]*len(order_by) + [True]
            needers = needers.sort_values(by=by, ascending=asc).reset_index(drop=False)
        else:
            needers = needers.sort_values(["store_code"]).reset_index(drop=False)

        # round-robin
        idx = 0
        n = len(needers)
        remaining = needers["ihtiyac_adet"].tolist()

        while avail > 0 and any(r > 0 for r in remaining):
            if remaining[idx] > 0:
                allocations.append({
                    "alt_yer": alt_yer,
                    "store_code": needers.loc[idx, "store_code"],
                    keys[0]: sku,
                    keys[1]: pid,
                    "alloc_qty": 1
                })
                remaining[idx] -= 1
                avail -= 1
                used_totals[(alt_yer, sku, pid)] += 1
            idx = (idx + 1) % n

        # dem’de kalan ihtiyacı güncelle
        dem.loc[needers["index"], "ihtiyac_adet"] = remaining

    # ---- çıktı tabloları ----
    if allocations:
        alloc_df = (pd.DataFrame(allocations)
                    .groupby(["alt_yer","store_code", *keys], as_index=False)["alloc_qty"].sum())
    else:
        alloc_df = pd.DataFrame(columns=["alt_yer","store_code", *keys, "alloc_qty"])

    wh_agg["allocated"] = wh_agg.apply(lambda r: used_totals.get((r["alt_yer"], r[keys[0]], r[keys[1]]), 0), axis=1)
    wh_agg["remaining"] = wh_agg["available"] - wh_agg["allocated"]

    remaining_need = dem.rename(columns={"ihtiyac_adet":"remaining_need"})[
        ["store_code", *keys, "remaining_need"]
    ]

    return alloc_df, wh_agg[["alt_yer", *keys, "available", "allocated", "remaining"]], remaining_need


In [38]:
demand_df = result[["store_code","stok_kodu","product_id","ihtiyac_adet"]].copy()
wh_df     = warehouse_stock[["alt_yer","stok_kodu","product_id","miktar"]].copy()

alloc, wh_after, need_after = allocate_round_robin(demand_df, wh_df)

In [39]:
demand_df[demand_df['product_id']=='205455-STDStandartSTD']

,store_code,stok_kodu,product_id,ihtiyac_adet
2247,1101,205455-STD,205455-STDStandartSTD,0
13587,1301,205455-STD,205455-STDStandartSTD,0
30489,1401,205455-STD,205455-STDStandartSTD,0
53008,1601,205455-STD,205455-STDStandartSTD,0
66942,1801,205455-STD,205455-STDStandartSTD,0
83033,1901,205455-STD,205455-STDStandartSTD,0
100844,2001,205455-STD,205455-STDStandartSTD,0
114318,201,205455-STD,205455-STDStandartSTD,0
126903,2101,205455-STD,205455-STDStandartSTD,22
147743,2301,205455-STD,205455-STDStandartSTD,0


In [40]:
wh_df[wh_df['product_id']=='205455-STDStandartSTD']

,alt_yer,stok_kodu,product_id,miktar
3223,900101,205455-STD,205455-STDStandartSTD,1036.0


In [41]:
alloc[alloc['product_id']=='205455-STDStandartSTD']

,alt_yer,store_code,stok_kodu,product_id,alloc_qty
503,900101,2101,205455-STD,205455-STDStandartSTD,22
671,900101,2401,205455-STD,205455-STDStandartSTD,6


In [42]:
wh_after[wh_after['product_id']=='205455-STDStandartSTD']

,alt_yer,stok_kodu,product_id,available,allocated,remaining
2621,900101,205455-STD,205455-STDStandartSTD,1036,28,1008


In [43]:
need_after[need_after['product_id']=='205455-STDStandartSTD']

,store_code,stok_kodu,product_id,remaining_need
126903,2101,205455-STD,205455-STDStandartSTD,0
165052,2401,205455-STD,205455-STDStandartSTD,0
